In [1]:
# !pip install meteostat

In [3]:
import pandas as pd
from meteostat import Point, Hourly
import sqlite3

In [9]:
df = pd.read_parquet("main_database/fhvhv_tripdata_2025-03.parquet")

In [10]:
df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,cbd_congestion_fee
0,HV0003,B03404,B03404,2025-03-01 00:02:25,2025-03-01 00:05:36,2025-03-01 00:05:53,2025-03-01 00:12:54,255,256,1.01,...,0.00,0.0,0.00,13.72,N,N,N,N,N,0.0
1,HV0003,B03404,B03404,2025-03-01 00:13:31,2025-03-01 00:16:20,2025-03-01 00:16:20,2025-03-01 00:42:33,256,43,6.06,...,2.75,0.0,8.63,36.96,N,N,N,N,N,1.5
2,HV0003,B03404,B03404,2025-03-01 00:02:04,2025-03-01 00:04:05,2025-03-01 00:04:54,2025-03-01 00:18:02,150,21,3.44,...,0.00,0.0,0.00,12.80,N,N,N,N,N,0.0
3,HV0003,B03404,B03404,2025-03-01 00:26:14,2025-03-01 00:28:06,2025-03-01 00:28:43,2025-03-01 00:54:39,11,83,14.78,...,0.00,0.0,0.00,36.58,N,N,N,N,N,0.0
4,HV0005,B03406,None,2025-03-01 00:23:35,NaT,2025-03-01 00:31:44,2025-03-01 00:43:42,203,203,1.53,...,0.00,0.0,0.00,11.40,N,N,N,N,Y,0.0


In [13]:
df.columns

Index(['hvfhs_license_num', 'dispatching_base_num', 'originating_base_num',
       'request_datetime', 'on_scene_datetime', 'pickup_datetime',
       'dropoff_datetime', 'PULocationID', 'DOLocationID', 'trip_miles',
       'trip_time', 'base_passenger_fare', 'tolls', 'bcf', 'sales_tax',
       'congestion_surcharge', 'airport_fee', 'tips', 'driver_pay',
       'shared_request_flag', 'shared_match_flag', 'access_a_ride_flag',
       'wav_request_flag', 'wav_match_flag', 'cbd_congestion_fee'],
      dtype='object')

In [15]:
df.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[us]
on_scene_datetime       datetime64[us]
pickup_datetime         datetime64[us]
dropoff_datetime        datetime64[us]
PULocationID                     int32
DOLocationID                     int32
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
cbd_congestion_fee             float64
dtype: object

In [17]:
df.shape

(20536879, 25)

In [19]:
df.isnull().sum()

hvfhs_license_num             0
dispatching_base_num          0
originating_base_num    5973083
request_datetime              0
on_scene_datetime        928830
pickup_datetime               0
dropoff_datetime              0
PULocationID                  0
DOLocationID                  0
trip_miles                    0
trip_time                     0
base_passenger_fare           0
tolls                         0
bcf                           0
sales_tax                     0
congestion_surcharge          0
airport_fee                   0
tips                          0
driver_pay                    0
shared_request_flag           0
shared_match_flag             0
access_a_ride_flag            0
wav_request_flag              0
wav_match_flag                0
cbd_congestion_fee            0
dtype: int64

In [20]:
df.drop_duplicates()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,cbd_congestion_fee
0,HV0003,B03404,B03404,2025-03-01 00:02:25,2025-03-01 00:05:36,2025-03-01 00:05:53,2025-03-01 00:12:54,255,256,1.010,...,0.00,0.0,0.00,13.72,N,N,N,N,N,0.0
1,HV0003,B03404,B03404,2025-03-01 00:13:31,2025-03-01 00:16:20,2025-03-01 00:16:20,2025-03-01 00:42:33,256,43,6.060,...,2.75,0.0,8.63,36.96,N,N,N,N,N,1.5
2,HV0003,B03404,B03404,2025-03-01 00:02:04,2025-03-01 00:04:05,2025-03-01 00:04:54,2025-03-01 00:18:02,150,21,3.440,...,0.00,0.0,0.00,12.80,N,N,N,N,N,0.0
3,HV0003,B03404,B03404,2025-03-01 00:26:14,2025-03-01 00:28:06,2025-03-01 00:28:43,2025-03-01 00:54:39,11,83,14.780,...,0.00,0.0,0.00,36.58,N,N,N,N,N,0.0
4,HV0005,B03406,None,2025-03-01 00:23:35,NaT,2025-03-01 00:31:44,2025-03-01 00:43:42,203,203,1.530,...,0.00,0.0,0.00,11.40,N,N,N,N,Y,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20536874,HV0003,B03404,B03404,2025-03-31 23:45:25,2025-03-31 23:46:37,2025-03-31 23:48:39,2025-04-01 00:00:43,126,213,2.330,...,0.00,0.0,2.04,12.93,N,N,N,N,N,0.0
20536875,HV0005,B03406,None,2025-03-31 23:00:07,2025-03-31 23:00:57,2025-03-31 23:01:57,2025-03-31 23:10:55,186,158,1.863,...,2.75,0.0,0.00,16.19,N,N,N,N,N,1.5
20536876,HV0003,B03404,B03404,2025-03-31 22:59:50,2025-03-31 23:11:33,2025-03-31 23:11:44,2025-03-31 23:28:15,239,263,2.970,...,2.75,0.0,0.00,17.25,N,N,N,N,N,0.0
20536877,HV0003,B03404,B03404,2025-03-31 23:05:58,2025-03-31 23:09:57,2025-03-31 23:10:43,2025-03-31 23:19:01,177,77,2.070,...,0.00,0.0,0.00,9.57,N,N,N,N,N,0.0


In [23]:
df["hvfhs_license_num"].unique()

array(['HV0003', 'HV0005'], dtype=object)

In [25]:
df[df["hvfhs_license_num"]=="HV0003"].shape

(14547181, 25)

In [27]:
uber_rides_df = df[df["hvfhs_license_num"] == "HV0003"]

In [29]:
df[df["hvfhs_license_num"]=="HV0005"].shape

(5989698, 25)

In [31]:
lyft_rides_df = df[df["hvfhs_license_num"] == "HV0005"]

In [33]:
uber_sample_df = uber_rides_df.sample(n = 10000, random_state = 1)

In [35]:
lyft_sample_df = lyft_rides_df.sample(n = 10000, random_state = 1)

In [37]:
# Combining uber and lyft datasets together
ride_data_df = pd.concat([uber_sample_df, lyft_sample_df], axis = 0)

# Resetting index
ride_data_df.reset_index(drop = True, inplace = True)

ride_data_df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,congestion_surcharge,airport_fee,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,cbd_congestion_fee
0,HV0003,B03404,B03404,2025-03-03 08:50:50,2025-03-03 08:51:10,2025-03-03 08:53:11,2025-03-03 09:04:35,161,162,0.57,...,2.75,0.0,0.0,7.70,N,N,N,N,N,1.5
1,HV0003,B03404,B03404,2025-03-30 10:01:05,2025-03-30 10:04:45,2025-03-30 10:05:02,2025-03-30 10:17:03,61,37,2.14,...,0.00,0.0,0.0,14.55,N,N,N,N,N,0.0
2,HV0003,B03404,B03404,2025-03-15 08:10:41,2025-03-15 08:14:21,2025-03-15 08:16:22,2025-03-15 08:20:29,200,220,0.92,...,0.00,0.0,0.0,6.00,N,N,N,N,Y,0.0
3,HV0003,B03404,B03404,2025-03-01 05:42:30,2025-03-01 05:55:15,2025-03-01 05:56:45,2025-03-01 06:38:43,249,177,7.57,...,2.75,0.0,0.0,40.38,N,N,N,N,N,1.5
4,HV0003,B03404,B03404,2025-03-09 01:51:57,2025-03-09 01:54:59,2025-03-09 01:57:01,2025-03-09 03:15:13,157,61,3.77,...,0.00,0.0,0.0,16.35,N,N,N,N,N,0.0


In [39]:
ride_data_df.to_csv("results/ride_data.csv", index = False)

# Adding weather data with to the dataset

In [41]:
# Loading the ride_data.csv
rides_df = pd.read_csv("results/ride_data.csv")

In [43]:
rides_df.dtypes

hvfhs_license_num        object
dispatching_base_num     object
originating_base_num     object
request_datetime         object
on_scene_datetime        object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
trip_miles              float64
trip_time                 int64
base_passenger_fare     float64
tolls                   float64
bcf                     float64
sales_tax               float64
congestion_surcharge    float64
airport_fee             float64
tips                    float64
driver_pay              float64
shared_request_flag      object
shared_match_flag        object
access_a_ride_flag       object
wav_request_flag         object
wav_match_flag           object
cbd_congestion_fee      float64
dtype: object

In [41]:
# Convert the 'request_datetime' column to datetime
rides_df["request_datetime"] = pd.to_datetime(rides_df["request_datetime"])

In [43]:
LATITUDE = 40.7128  # NYC latitude
LONGITUDE = -74.0060  # NYC longitude

In [45]:
# Function to get weather data using Meteostat
async def get_weather_data(lat, lon, start_date, end_date):
    point = Point(lat, lon)
    weather_data = Hourly(point, start_date, end_date)
    return weather_data.fetch()

In [47]:
# Function to fetch weather data for each request_datetime
async def fetch_weather_data():
    weather_data = []
    for index, row in rides_df.iterrows():
        date_time = row['request_datetime']  # Get the specific timestamp
        # Round the timestamp to the nearest hour
        rounded_time = date_time.round('h')
        print(f"Fetching weather data for: {rounded_time}")  # Debug print
        
        # Fetch weather data for the rounded date and hour
        weather_info = await get_weather_data(LATITUDE, LONGITUDE, rounded_time, rounded_time)
        
        if weather_info.empty:
            print(f"No weather data found for: {rounded_time}")  # Debug print for empty data
        else:
            print(f"Weather data fetched for: {rounded_time}")  # Debug print for successful fetch
        
        weather_data.append(weather_info)
    
    return weather_data

In [49]:
# Call the function to fetch weather data
weather_data = await fetch_weather_data()

Fetching weather data for: 2025-03-03 09:00:00
Weather data fetched for: 2025-03-03 09:00:00
Fetching weather data for: 2025-03-30 10:00:00
Weather data fetched for: 2025-03-30 10:00:00
Fetching weather data for: 2025-03-15 08:00:00
Weather data fetched for: 2025-03-15 08:00:00
Fetching weather data for: 2025-03-01 06:00:00
Weather data fetched for: 2025-03-01 06:00:00
Fetching weather data for: 2025-03-09 02:00:00
Weather data fetched for: 2025-03-09 02:00:00
Fetching weather data for: 2025-03-07 22:00:00
Weather data fetched for: 2025-03-07 22:00:00
Fetching weather data for: 2025-03-08 21:00:00
Weather data fetched for: 2025-03-08 21:00:00
Fetching weather data for: 2025-03-25 13:00:00
Weather data fetched for: 2025-03-25 13:00:00
Fetching weather data for: 2025-03-05 15:00:00
Weather data fetched for: 2025-03-05 15:00:00
Fetching weather data for: 2025-03-08 18:00:00
Weather data fetched for: 2025-03-08 18:00:00
Fetching weather data for: 2025-03-22 16:00:00
Weather data fetched fo

In [51]:
# Add weather data to the original dataset
rides_df['weather_data'] = weather_data

In [53]:
rides_df["weather_data"][0]

,temp,dwpt,rhum,prcp,snow,wdir,wspd,wpgt,pres,tsun,coco
time,,,,,,,,,,,
2025-03-03 09:00:00,-3.9,-12.9,55.0,0.0,<NA>,294.0,7.6,<NA>,1024.3,<NA>,1.0


In [55]:
# Define the mapping of weather condition codes to descriptions
weather_condition_mapping = {
    1: "Clear",
    2: "Fair",
    3: "Cloudy",
    4: "Overcast",
    5: "Fog",
    6: "Freezing Fog",
    7: "Light Rain",
    8: "Rain",
    9: "Heavy Rain",
    10: "Freezing Rain",
    11: "Heavy Freezing Rain",
    12: "Sleet",
    13: "Heavy Sleet",
    14: "Light Snowfall",
    15: "Snowfall",
    16: "Heavy Snowfall",
    17: "Rain Shower",
    18: "Heavy Rain Shower",
    19: "Sleet Shower",
    20: "Heavy Sleet Shower",
    21: "Snow Shower",
    22: "Heavy Snow Shower",
    23: "Lightning",
    24: "Hail",
    25: "Thunderstorm",
    26: "Heavy Thunderstorm",
    27: "Storm"
}

In [57]:
# Create a new column in rides_df for weather conditions
rides_df["weather_condition"] = None

In [59]:
# Iterate through the rides_df and assign weather conditions based on coco values
for index, row in rides_df.iterrows():
    # Get the weather data for the specific ride
    weather_info = row["weather_data"]
    
    # Check if weather_info is not empty and has a valid index
    if not weather_info.empty:
        # Extract the coco value from the weather data
        coco_value = weather_info["coco"].iloc[0]  # Get the first (and likely only) value
        
        # Map the coco value to the weather condition description
        if coco_value in weather_condition_mapping:
            rides_df.at[index, "weather_condition"] = weather_condition_mapping[coco_value]
        else:
            rides_df.at[index, "weather_condition"] = "Unknown"  # Handle cases where coco value is not in the mapping
    else:
        rides_df.at[index, "weather_condition"] = "No Data"  # Handle cases where weather data is empty

# Now rides_df will have a new column "weather_condition" with the corresponding descriptions

In [61]:
rides_df.head(5)

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,cbd_congestion_fee,weather_data,weather_condition
0,HV0003,B03404,B03404,2025-03-03 08:50:50,2025-03-03 08:51:10,2025-03-03 08:53:11,2025-03-03 09:04:35,161,162,0.57,...,0.0,7.70,N,N,N,N,N,1.5,temp dwpt rhum prcp s...,Clear
1,HV0003,B03404,B03404,2025-03-30 10:01:05,2025-03-30 10:04:45,2025-03-30 10:05:02,2025-03-30 10:17:03,61,37,2.14,...,0.0,14.55,N,N,N,N,N,0.0,temp dwpt rhum prcp s...,Cloudy
2,HV0003,B03404,B03404,2025-03-15 08:10:41,2025-03-15 08:14:21,2025-03-15 08:16:22,2025-03-15 08:20:29,200,220,0.92,...,0.0,6.00,N,N,N,N,Y,0.0,temp dwpt rhum prcp s...,Overcast
3,HV0003,B03404,B03404,2025-03-01 05:42:30,2025-03-01 05:55:15,2025-03-01 05:56:45,2025-03-01 06:38:43,249,177,7.57,...,0.0,40.38,N,N,N,N,N,1.5,temp dwpt rhum prcp s...,Cloudy
4,HV0003,B03404,B03404,2025-03-09 01:51:57,2025-03-09 01:54:59,2025-03-09 01:57:01,2025-03-09 03:15:13,157,61,3.77,...,0.0,16.35,N,N,N,N,N,0.0,temp dwpt rhum prcp s...,Clear


In [71]:
rides_df["weather_condition"].unique()

array(['Clear', 'Cloudy', 'Overcast', 'Rain', 'Fair', 'Light Rain',
       'Heavy Rain', 'Fog', 'Rain Shower', 'Heavy Rain Shower'],
      dtype=object)

In [73]:
rides_df.to_csv("results/ride_data_with_weather_condition.csv", index = False)

# Creating a SQLite database from CSV file

In [45]:
ride_sharing_df = pd.read_csv("results/ride_data_with_weather_condition.csv")

In [47]:
ride_sharing_df.head()

,hvfhs_license_num,dispatching_base_num,originating_base_num,request_datetime,on_scene_datetime,pickup_datetime,dropoff_datetime,PULocationID,DOLocationID,trip_miles,...,tips,driver_pay,shared_request_flag,shared_match_flag,access_a_ride_flag,wav_request_flag,wav_match_flag,cbd_congestion_fee,weather_data,weather_condition
0,HV0003,B03404,B03404,2025-03-03 08:50:50,2025-03-03 08:51:10,2025-03-03 08:53:11,2025-03-03 09:04:35,161,162,0.57,...,0.0,7.70,N,N,N,N,N,1.5,temp dwpt rhum prcp s...,Clear
1,HV0003,B03404,B03404,2025-03-30 10:01:05,2025-03-30 10:04:45,2025-03-30 10:05:02,2025-03-30 10:17:03,61,37,2.14,...,0.0,14.55,N,N,N,N,N,0.0,temp dwpt rhum prcp s...,Cloudy
2,HV0003,B03404,B03404,2025-03-15 08:10:41,2025-03-15 08:14:21,2025-03-15 08:16:22,2025-03-15 08:20:29,200,220,0.92,...,0.0,6.00,N,N,N,N,Y,0.0,temp dwpt rhum prcp s...,Overcast
3,HV0003,B03404,B03404,2025-03-01 05:42:30,2025-03-01 05:55:15,2025-03-01 05:56:45,2025-03-01 06:38:43,249,177,7.57,...,0.0,40.38,N,N,N,N,N,1.5,temp dwpt rhum prcp s...,Cloudy
4,HV0003,B03404,B03404,2025-03-09 01:51:57,2025-03-09 01:54:59,2025-03-09 01:57:01,2025-03-09 03:15:13,157,61,3.77,...,0.0,16.35,N,N,N,N,N,0.0,temp dwpt rhum prcp s...,Clear


In [49]:
ride_sharing_df = ride_sharing_df.drop("weather_data", axis = 1)

In [51]:
ride_sharing_df.dtypes

hvfhs_license_num        object
dispatching_base_num     object
originating_base_num     object
request_datetime         object
on_scene_datetime        object
pickup_datetime          object
dropoff_datetime         object
PULocationID              int64
DOLocationID              int64
trip_miles              float64
trip_time                 int64
base_passenger_fare     float64
tolls                   float64
bcf                     float64
sales_tax               float64
congestion_surcharge    float64
airport_fee             float64
tips                    float64
driver_pay              float64
shared_request_flag      object
shared_match_flag        object
access_a_ride_flag       object
wav_request_flag         object
wav_match_flag           object
cbd_congestion_fee      float64
weather_condition        object
dtype: object

In [53]:
# Convert the necessary columns to datetime format
ride_sharing_df["request_datetime"] = pd.to_datetime(ride_sharing_df["request_datetime"])
ride_sharing_df["on_scene_datetime"] = pd.to_datetime(ride_sharing_df["on_scene_datetime"])
ride_sharing_df["pickup_datetime"] = pd.to_datetime(ride_sharing_df["pickup_datetime"])
ride_sharing_df["dropoff_datetime"] = pd.to_datetime(ride_sharing_df["dropoff_datetime"])

In [55]:
ride_sharing_df.dtypes

hvfhs_license_num               object
dispatching_base_num            object
originating_base_num            object
request_datetime        datetime64[ns]
on_scene_datetime       datetime64[ns]
pickup_datetime         datetime64[ns]
dropoff_datetime        datetime64[ns]
PULocationID                     int64
DOLocationID                     int64
trip_miles                     float64
trip_time                        int64
base_passenger_fare            float64
tolls                          float64
bcf                            float64
sales_tax                      float64
congestion_surcharge           float64
airport_fee                    float64
tips                           float64
driver_pay                     float64
shared_request_flag             object
shared_match_flag               object
access_a_ride_flag              object
wav_request_flag                object
wav_match_flag                  object
cbd_congestion_fee             float64
weather_condition        

In [57]:
# Create a SQLite database
conn = sqlite3.connect("results/ride_share.db")

In [59]:
# Write the data to a SQLite table
ride_sharing_df.to_sql('ride_share_table', conn, if_exists='replace', index=False)

20000

In [61]:
# Close the connection
conn.close()

In [63]:
# Connect to the SQLite database
conn = sqlite3.connect("results/ride_share.db")


In [65]:
table_name = 'ride_share_table'
cursor = conn.cursor()
cursor.execute(f"PRAGMA table_info({table_name});")
columns_info = cursor.fetchall()


In [67]:
# Print the column names and data types
print("Column Name | Data Type")
print("-------------------------")
for column in columns_info:
    column_name = column[1]  # Column name is the second element
    data_type = column[2]     # Data type is the third element
    print(f"{column_name} | {data_type}")


Column Name | Data Type
-------------------------
hvfhs_license_num | TEXT
dispatching_base_num | TEXT
originating_base_num | TEXT
request_datetime | TIMESTAMP
on_scene_datetime | TIMESTAMP
pickup_datetime | TIMESTAMP
dropoff_datetime | TIMESTAMP
PULocationID | INTEGER
DOLocationID | INTEGER
trip_miles | REAL
trip_time | INTEGER
base_passenger_fare | REAL
tolls | REAL
bcf | REAL
sales_tax | REAL
congestion_surcharge | REAL
airport_fee | REAL
tips | REAL
driver_pay | REAL
shared_request_flag | TEXT
shared_match_flag | TEXT
access_a_ride_flag | TEXT
wav_request_flag | TEXT
wav_match_flag | TEXT
cbd_congestion_fee | REAL
weather_condition | TEXT


In [69]:
# Define the SQL query
query = """
SELECT * FROM ride_share_table
WHERE DATE(pickup_datetime) = '2025-03-01';
"""

# Execute the query
cursor.execute(query)


In [71]:
# Fetch all results
results = cursor.fetchall()

# Print the results
for row in results:
    print(row)


('HV0003', 'B03404', 'B03404', '2025-03-01 05:42:30', '2025-03-01 05:55:15', '2025-03-01 05:56:45', '2025-03-01 06:38:43', 249, 177, 7.57, 2518, 40.87, 0.0, 1.01, 3.73, 2.75, 0.0, 0.0, 40.38, 'N', 'N', 'N', 'N', 'N', 1.5, 'Cloudy')
('HV0003', 'B03404', 'B03404', '2025-03-01 02:18:35', '2025-03-01 02:19:20', '2025-03-01 02:22:03', '2025-03-01 02:37:04', 148, 249, 2.19, 902, 31.9, 0.0, 0.76, 2.83, 2.75, 0.0, 0.0, 20.97, 'N', 'N', 'N', 'N', 'N', 1.5, 'Cloudy')
('HV0003', 'B03404', 'B03404', '2025-03-01 10:41:35', '2025-03-01 10:43:16', '2025-03-01 10:43:27', '2025-03-01 11:08:24', 228, 189, 6.28, 1498, 34.22, 0.0, 0.86, 3.04, 0.0, 0.0, 0.0, 23.98, 'N', 'N', 'N', 'N', 'N', 0.0, 'Cloudy')
('HV0003', 'B03404', 'B03404', '2025-03-01 12:47:29', '2025-03-01 12:47:57', '2025-03-01 12:49:57', '2025-03-01 13:03:04', 114, 232, 1.39, 787, 16.14, 0.0, 0.37, 1.44, 2.75, 0.0, 0.0, 9.88, 'N', 'N', 'N', 'N', 'N', 1.5, 'Cloudy')
('HV0003', 'B03404', 'B03404', '2025-03-01 14:03:32', '2025-03-01 14:04:58', 

In [73]:
# Close the connection
conn.close()